This Jupyter Notebook contains code to do the following:
- Clean EAGLE-I Data
- Extract County Events from EAGLE-I Data
- Extract Multi-County events based on space and time

The notebook is divided into sections and each code block is suplemented with comments and markdown text to guide the user about whats going on.

All or parts of the code from this notebook can be easily copied to create standalone python script files.
Further information about the methods used in this notebook can be found in the following papers:
- https://arxiv.org/abs/2511.12685
- 


# Initialization

In [1]:
# importing necessary libraries
import os
import sys
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from src.eaglei_modules.eagleiEventProcessing import verify_eaglei_files, clean_eaglei_state_data

In [3]:
# verify that EAGLEi files are present
eaglei_files = verify_eaglei_files()

Found EAGLEi outage data files for 11 years: 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024


In [4]:
state = 'Florida'

eaglei_state_data = clean_eaglei_state_data(state)

Cleaned data for Florida already exists as eaglei_cleaned_florida.parquet. Loading the existing file...


In [5]:
eaglei_state_data.head()

,fips_code,county,state,run_start_time,total_customers,customers_out
0,12011,Broward,Florida,2014-11-01 04:00:00,NaN,17
1,12021,Collier,Florida,2014-11-01 04:00:00,NaN,14
2,12069,Lake,Florida,2014-11-01 04:00:00,NaN,1
3,12071,Lee,Florida,2014-11-01 04:00:00,NaN,1
4,12085,Martin,Florida,2014-11-01 04:00:00,NaN,12
